# Kubernetes

Dados la problemática que presenta la escabilidad en entornos de múltiples servidores, se creó Kubernetes para permitir una administración de estos, permitiendo automatizar el despliegue, ajustar la escala y ayudar con el manejo de contenedores

### Un ejemplo

Si durante la noche un servidor sufre un desperfecto, kubernetes se encargará de montar los contenedores perdidos en otros servidores para mantener el sistema funcionando

Kubernetes tiene tres partes muy importantes:
- Controladores: Se preocupan de decidir donde estarán los contenedores, los requerimientos que necesita cada uno
- Workers: Máquinas que ejecutan pods, los nodos trabajan en red, es decir, si un nodo cae no importa, el controlador puede levantar los pods perdidos en otra máquina y la red continúa trabajando.
- etcd: Aquí se almacena la información del estado de elemento miembro del clúster.


![title](images/arquitectura)

# Montando un clúster básico con Minikube



Chequear que la virtualización está habilitada en la bios del pc:  
```egrep --color 'vmx|svm' /proc/cpuinfo```

Instalar un operador de maquinas virtuales (VirtualBox, KVM, etc)  

Instalar kubectl:
```sudo pacman -S kubectl```

Instalar minikube:
```sudo pacman -S minikube```


In [6]:
minikube start --nodes 2 -p multinode-demo

😄  [multinode-demo] minikube v1.13.1 en Arch 20.1.1
✨  Using the kvm2 driver based on existing profile
👍  Starting control plane node multinode-demo in cluster multinode-demo
🏃  Updating the running kvm2 "multinode-demo" VM ...
🎉  minikube 1.14.0 is available! Download it: https://github.com/kubernetes/minikube/releases/tag/v1.14.0
💡  To disable this notice, run: 'minikube config set WantUpdateNotification false'

🐳  Preparando Kubernetes v1.19.2 en Docker 19.03.12...
🔗  Configuring CNI (Container Networking Interface) ...
🔎  Verifying Kubernetes components...
🌟  Enabled addons: default-storageclass, storage-provisioner
❗  The cluster multinode-demo already exists which means the --nodes parameter will be ignored. Use "minikube node add" to add nodes to an existing cluster.
👍  Starting node multinode-demo-m02 in cluster multinode-demo
🏃  Updating the running kvm2 "multinode-demo-m02" VM ...
🌐  Se han encontrado las siguientes opciones de red:
    ▪ NO_PROXY=192.168.39.15
🐳  Preparando 

In [7]:
kubectl get nodes

NAME                 STATUS   ROLES    AGE   VERSION
multinode-demo       Ready    master   16m   v1.19.2
multinode-demo-m02   Ready    <none>   10m   v1.19.2


In [8]:
minikube status -p multinode-demo

multinode-demo
type: Control Plane
host: Running
kubelet: Running
apiserver: Running
kubeconfig: Configured

multinode-demo-m02
type: Worker
host: Running
kubelet: Running



## Conceptos



### Namespace  
Básicamente crean clústers virtuales dentro del mismo clúster físico, ocupan la misma infraestructura pero cada uno permanece aislado del otro, por ejemplo esto es ideal para tener clústers de desarrollo y producción.  

In [12]:
kubectl get namespaces

NAME              STATUS   AGE
default           Active   72m
kube-node-lease   Active   72m
kube-public       Active   72m
kube-system       Active   72m


### Pods  

Es la unidad mas pequeña que maneja kubernetes, consiste en un conjunto de contenedores donde funciona alguna aplicación.  

#### Consideraciones
No use Pods desnudos (es decir, Pods que no estén vinculados a ReplicaSet o Deployment) si puede evitarlo. Los pods desnudos no se reprogramarán en caso de falla de un nodo.  

Una deployment crea un ReplicaSet para garantizar que la cantidad deseada de pods siempre esté disponible, y especifica una estrategia para reemplazar los pods (como RollingUpdate), es casi siempre preferible a crear pods directamente. Un job también puede ser apropiado.

In [13]:
kubectl get pods

No resources found in default namespace.


### Servicios  

Es una abstracción que define un conjunto de pods que implementan un micro-servicio.  

Existen 3 tipos de servicios:



ClusterIP:  
Solo permite el acceso interno entre distintos servicios. Es el tipo por defecto. Podemos acceder desde el exterior con la instrucción kubectl proxy, puede de ser gran ayuda para los desarrolladores.  

![title](images/ClusterIP)

NodePort:  
Abre un puerto, para que el servicio sea accesible desde el exterior. Por defecto el puerto generado está en el rango de 30000:40000. Para acceder usamos la ip del servidor master del cluster y el puerto asignado.

![title](images/NodePort)

LoadBalancer:  
Este tipo sólo esta soportado en servicios de cloud público (GKE, AKS o AWS). El proveedor asignara un recurso de balanceo ed carga para el acceso a los servicios. si usamos un cloud privado, como OpenSatck necesitaremos un plugin para configurar el funcionamiento.

![title](images/LoadBalancer)

#### Consideraciones
Define y usa etiquetas que identifiquen atributos semanticos de tu aplicación o deployment, por ejemplo:
```{ app: myapp, tier: frontend, phase: test, deployment: v3 }```. Puedes usar estas etiquetas para seleccionar apropiadamente los pods para otros recursos; por ejemplo, si quiero que un servicio seleccione todos los pods ```tier: frontend``` o todos los componentes ```phase: test``` de la ```app: myapp```.

In [14]:
kubectl get services

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   75m


### Manifiestos  
Son documentos utilizados para crear, modificar o eliminar recursos de Kubernetes, como pods, despliegues, servicios e ingresos.


In [14]:
cat yaml/00-namespace.yaml

kind: Namespace
apiVersion: v1
metadata:
  name: testing 


In [1]:
kubectl apply -f 00-namespace.yaml

namespace/testing created


In [2]:
kubectl get namespaces

NAME              STATUS   AGE
default           Active   137m
kube-node-lease   Active   137m
kube-public       Active   137m
kube-system       Active   137m
testing           Active   24s


In [17]:
cat yaml/01-wordpress-service.yaml

kind: Service           #pod, deployment, service, namespace, replicaset, StatefulSet, DaemonSet, 
                        #Job o lo que sea que vayas a describir
apiVersion: v1
metadata:
  name: wordpress       #nombre del servicio en este caso
spec:
  type: NodePort        #Aquí se indica si se trabajará con las ip internas o se abrirá un puerto 
                        #para desplegarlo externamente, según vimos, NodePort ordena abrir un puerto
                        #en el worker para poder acceder externamente a este servicio
  ports:
  - port:80             #Se indica el puerto que se abre en el worker
    targetPort: 80      #Es el puerto al que apunta el servicio en cualquier pod que tenga el rol
                        #wordpress
    nodePort: 30000     #Es el puerto del nodo que apunta al 80 del pod
  selector:
    role: wordpress

# Si se desea, se puede especificar otro recurso en el manifiesto separandolos por guiones
#---


In [20]:
kubectl -n testing apply -f yaml/01-wordpress-service.yaml
kubectl -n testing get services

service/wordpress created
NAME        TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
wordpress   NodePort   10.108.139.33   <none>        80:30000/TCP   1s


In [21]:
cat yaml/02-wordpress-rc.yaml

apiVersion: v1
kind: ReplicationController
metadata:
    name: wordpress
spec:
    replicas: 1
    template:
        metadata:
            labels:
                role: wordpress
        spec:
            containers:
            - name: wordpress
                image: wordpress:php7.1-apache
                imagePullPolicy: IfNotPresent
                ports:
                - containerPort: 80


La imagen se descarga por defecto desde DockerHub; si quieres usar un registry alternativo, debes indicar la URL completa al recurso.

In [25]:
kubectl -n testing apply -f yaml/02-wordpress-rc.yaml
kubectl -n testing get pods

replicationcontroller/wordpress created
NAME              READY   STATUS    RESTARTS   AGE
wordpress-sjjwt   0/1     Pending   0          0s


In [26]:
kubectl -n testing delete pod wordpress-sjjwt
kubectl -n testing get pods

pod "wordpress-sjjwt" deleted
NAME              READY   STATUS    RESTARTS   AGE
wordpress-mqr2h   1/1     Running   0          41s
